### testing with the faust

In [1]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_groq import ChatGroq
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_community.graphs import Neo4jGraph
import streamlit as st


NEO4J_URI = st.secrets["NEO4J_URI"]
NEO4J_USERNAME = st.secrets["NEO4J_USERNAME"]
NEO4J_PASSWORD = st.secrets["NEO4J_PASSWORD"]
NEO4J_DATABASE = st.secrets["NEO4J_DATABASE"]

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

rate_limiter = InMemoryRateLimiter(
    requests_per_second=1,
    max_bucket_size=10687,
)

llm = ChatGroq(
    temperature=0.0, 
    model_name="llama3-groq-70b-8192-tool-use-preview",
    rate_limiter=rate_limiter,
    api_key=st.secrets["GROQ_API_KEY"]
)

# allowed_nodes = ['subject', 'object']
# allowed_relationships = ['SPEAKS_TO', 'DIALOGE_FOLLOWED_BY']

llm_transformer = LLMGraphTransformer(
    llm=llm,
    # allowed_nodes=allowed_nodes,
    # allowed_relationships=allowed_relationships,
    strict_mode=True,
    node_properties = True,
    relationship_properties = True,
    ignore_tool_usage = False
)

C:\Users\makos2tamas911\AppData\Local\Temp\ipykernel_15672\1832367587.py:20: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


In [11]:
text =  """
Mephistopheles and Faust are standing near a gathering of people, where flames are rising and witches are present. Mephistopheles comments on the lively atmosphere, but notes that in such a place, one cannot be alone. Faust expresses a desire to be at the top of the gathering, where he sees whirling smoke and thousands of people flocking to the evil spirit. Mephistopheles responds that the evil spirit will only create more problems, but suggests that they can find quiet in the midst of the chaos. He points out young and old witches, some of whom are veiled, and promises Faust that he can experience great joy with little effort.
As they prepare to enter the gathering, Faust asks Mephistopheles how he intends to introduce himself, as a devil or conjurer. Mephistopheles prefers to remain incognito but may show his true nature on special occasions. He teases an old woman who is cautious around him, sensing that she has detected something hellish about him.
Mephistopheles and Faust approach a group of people sitting around a fire, including a General, Minister, Parvenu, and Author, who are discussing the changing times and how the world is becoming more shallow. Mephistopheles comments that the people seem ripe for doom and that the world is on its last legs.
A vendor of curious trinkets appears, showcasing items that have caused harm and mischief. Mephistopheles dismisses her wares as outdated and suggests that she should deal in newer, more exciting items. Faust is overwhelmed by the sights and sounds of the gathering, comparing it to a fair that surpasses Leipzig.
Mephistopheles points out a woman named Lilith, Adam's first wife, warning Faust to beware of her beautiful but treacherous hair. Faust notices an old and young woman sitting on the grass, who seem to have been dancing. Mephistopheles encourages Faust to join in the revelry.
As they dance, Faust shares a dream he had about an apple tree with two lovely apples, and the Young Witch responds by referencing the story of Adam and Eve. Mephistopheles shares a contrasting dream about a cloven tree with an ugly owl, and the Old Witch responds in kind. The Young Witch asks about Faust's presence, and he explains that Mephistopheles is everywhere, critiquing every step of the dance.
Finally, a Proctophantasmist appears, lamenting the persistence of phantoms and delusions in the enlightened age.
"""

In [12]:
from langchain_core.documents import Document
try:
    documents = [Document(page_content=text)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(graph_documents)
    print(f"Nodes:{graph_documents[0].nodes}")
    print(f"Relationships:{graph_documents[0].relationships}")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    print("Please check the input text and ensure it's suitable for graph conversion.")

Nodes:[Node(id='Mephistopheles', type='Person', properties={}), Node(id='Faust', type='Person', properties={}), Node(id='General', type='Person', properties={}), Node(id='Minister', type='Person', properties={}), Node(id='Parvenu', type='Person', properties={}), Node(id='Author', type='Person', properties={}), Node(id='Vendor', type='Person', properties={}), Node(id='Lilith', type='Person', properties={}), Node(id='Young Witch', type='Person', properties={}), Node(id='Old Witch', type='Person', properties={}), Node(id='Proctophantasmist', type='Person', properties={})]
Relationships:[Relationship(source=Node(id='Mephistopheles', type='Person', properties={}), target=Node(id='Faust', type='Person', properties={}), type='COMPANION', properties={}), Relationship(source=Node(id='Mephistopheles', type='Person', properties={}), target=Node(id='General', type='Person', properties={}), type='OBSERVER', properties={}), Relationship(source=Node(id='Mephistopheles', type='Person', properties={}),

You are a theatre play analyzer. I will provide you with a list of dialogues between characters. Your task is to:

Describe the current frame with an objective paragraph about what is happening.
Only use the information provided in the user input; do not use any additional knowledge or external information.
Do not include any analysis, interpretation, or personal opinions beyond the provided text.
Example Input:

[
  {
    "character": "Margaret",
    "dialogue": "Art thou a man? feel for my hapless plight.",
    "act": "throwing herself at his feet"
  },
  {
    "character": "Faust",
    "dialogue": "Thy cries will wake the watchers of the night!"
  },
  {
    "character": "Margaret",
    "dialogue": "Who gave thee, hangman, such a power To drag me from my cell at midnight hour? Have pity on me! Be not so harsh—so rough! Surely tomorrow morn is soon enough. [She stands up.] So young, so very young, am I, And must already die! Once I was lovely too—'twas this that caused my fall. Near was the friend, but far from me today; Torn lies the wreath, the flowers are scattered all. Oh tear me not so forcibly away! Spare me! what have I done to injure thee? Oh hear my prayer! for once compassion show! 'Tis the first time I ever looked on thee.",
    "act": "on her knees"
  }
]

Expected Output:

Margaret throws herself at Faust's feet, pleading with him by asking if he is a man and to feel pity for her unfortunate situation. Faust warns her that her cries might wake the watchers of the night. Margaret questions who gave him the power to drag her from her cell at midnight and begs him to have pity and not be so harsh. She suggests that waiting until morning would be enough. Standing up, she laments her youth and the prospect of dying so soon, reflecting on her lost beauty which led to her downfall. She mentions that a friend was once near but is now distant, using the imagery of a torn wreath and scattered flowers to describe her state. Kneeling again, she implores him not to take her away forcibly, asks what she has done to harm him, and pleads for compassion, noting that this is the first time she has ever seen him.

In [29]:
import json
import re
import io

def parse_faust_text(file_path):
    with io.open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    lines = text.splitlines()
    

    structured_data = []
    act = None
    scene = None
    i = 0

    while i < len(lines):
        line = lines[i].rstrip('\n')  # Remove only the newline character at the end

        # Detect Act
        act_match = re.match(r'^ACT ([IVX]+)\.$', line)
        if act_match:
            act_number = act_match.group(1)
            act = {'act': act_number, 'scenes': []}
            structured_data.append(act)
            scene = None  # Reset scene
            i += 1
            continue

        # Detect Scene
        scene_match = re.match(r'^Scene ([IVX]+)\.$', line)
        if scene_match:
            scene_number = scene_match.group(1)
            scene = {'scene': scene_number, 'events': []}
            if act is not None:
                act['scenes'].append(scene)
            else:
                # Handle the case where a Scene appears before an Act
                act = {'act': 'Unknown', 'scenes': [scene]}
                structured_data.append(act)
            i += 1
            continue

        # Handle Empty Line
        if line.strip() == '':
            i += 1
            continue

        # Detect Character Name with possible stage direction
        # Do not strip leading spaces to ensure we only match lines starting at the beginning
        char_match = re.match(r'^([A-Za-z\s]+)\.\s*(\[_.*\])?$', line)
        if char_match:
            character = char_match.group(1).strip()
            act_description = ''
            # Check if there's a stage direction on the same line
            if char_match.group(2):
                act_description = char_match.group(2).replace('[_', '').replace(']', '').replace('_', '').strip()
            i += 1
            # Check for stage direction on the next line
            if not act_description and i < len(lines):
                next_line = lines[i].strip()
                if next_line.startswith('[_') and next_line.endswith(']'):
                    act_description = next_line.replace('[_', '').replace(']', '').replace('_', '').strip()
                    i += 1
            # Collect dialogue lines (lines that are indented)
            dialogue = ''
            while i < len(lines) and lines[i].strip() != '' and lines[i].startswith(' '):
                dialogue += lines[i].strip() + ' '
                i += 1
            # Store the event
            if scene is None:
                scene = {'scene': 'Unknown', 'events': []}
                if act is None:
                    act = {'act': 'Unknown', 'scenes': [scene]}
                    structured_data.append(act)
                else:
                    act['scenes'].append(scene)
            event = {'character': character + '.'}
            if dialogue.strip():
                event['dialogue'] = dialogue.strip().replace('_', '').replace('-', '')
            if act_description:
                event['act'] = act_description
            scene['events'].append(event)
            continue

        # Detect Stage Direction
        stage_direction_match = re.match(r'^\[(.*)\]$', line.strip())
        if stage_direction_match:
            stage_direction = stage_direction_match.group(1)
            if scene is None:
                scene = {'scene': 'Unknown', 'events': []}
                if act is None:
                    act = {'act': 'Unknown', 'scenes': [scene]}
                    structured_data.append(act)
                else:
                    act['scenes'].append(scene)
            scene['events'].append({'stage_direction': stage_direction.replace('[_', '').replace(']', '').replace('_', '')})
            i += 1
            continue

        # Handle other indented lines
        if line.startswith(' '):
            # Skip or handle as needed
            i += 1
            continue

        # Move to next line if none of the above matched
        i += 1

    # Set ensure_ascii=False to prevent escaping non-ASCII characters
    return json.dumps(structured_data, indent=2, ensure_ascii=False)

# Example usage:
structured_data = parse_faust_text(r'text\faust.txt')
print(structured_data)


[
  {
    "act": "Unknown",
    "scenes": [
      {
        "scene": "Unknown",
        "events": [
          {
            "character": "Scene I."
          },
          {
            "character": "Night."
          },
          {
            "character": "Faust.",
            "dialogue": "There now, I’ve toiled my way quite through Law, Medicine, and Philosophy, My bursting heart! And feelings, strange and new, At once through all my ravished senses dart! I feel my inmost soul made thrall to thee! Thou must! thou must! and were my life the fee!"
          },
          {
            "character": "Spirit.",
            "dialogue": "Who calls me?"
          },
          {
            "character": "Faust.",
            "dialogue": "Vision of affright!",
            "act": "turning away"
          },
          {
            "character": "Spirit.",
            "dialogue": "Thou hast with mighty spells invoked me, And to obey thy call provoked me, And now"
          },
          {
         